# **Promedio de precios por Segmento**


In [33]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    if nombre.startswith("df_")  and "puebla" in nombre and 'alfa' in nombre and 'df_alfa_q_puebla' not in nombre and '2024' in nombre and 'financiamientos' not in nombre and 'grupos' not in nombre and 'salarios' not in nombre and 'publicacion' not in nombre and 'df_feb_2024_puebla' not in nombre    #if nombre.startswith('df_') and 'alfa' in nombre and'puebla' in nombre and 'q' not in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_agosto_2024_puebla',
 'df_alfa_febrero_2024_puebla',
 'df_alfa_julio_2024_puebla',
 'df_alfa_junio_2024_puebla',
 'df_alfa_septiembre_2024_puebla']

In [34]:
#ers = df_may_2024_puebla[df_may_2024_puebla['categoria'].str.contains('E')]
#ers['precio'].unique()

Filtrar registros E con precios altos

In [35]:
# Filtrar los registros del segmento 'E'
for df in dataframes_list:
    segmento_e = df[df['segmento'] == 'E']  # Filtrar registros donde el segmento es 'E'

    # Opcional: ver los registros con los precios más altos
    top_precios_e = segmento_e.sort_values(by='precio', ascending=False).head(10)  # Los 10 registros con precios más altos
    print(top_precios_e[['categoria', 'precio']])  # Puedes ajustar las columnas a mostrar
1000000

     categoria       precio
3475        EL  335410000.0
3474        EL  135000000.0
3473        EL   60000000.0
3472        EL   60000000.0
3471        EL   55000000.0
3470        EL   45000000.0
3469        EL   41000000.0
3468        EL   40000000.0
3467        EL   40000000.0
3466        EL   39000000.0
     categoria    precio
3711        EL  60000000
3710        EL  40000000
3709        EL  37000000
3708        EL  35000000
3707        EL  34800000
3706        EL  34800000
3705        EL  34800000
3704        EL  33000000
3703        EL  32000000
3702        EL  31674000
     categoria       precio
4591        EL  335410000.0
4231        EL  152315600.0
4698        EL  135000000.0
4696        EL   60000000.0
4697        EL   60000000.0
4695        EL   55000000.0
4694        EL   45000000.0
4693        EL   41000000.0
4692        EL   40000000.0
4691        EL   40000000.0
     categoria        precio
4081        EL  1.520068e+08
4080        EL  1.350000e+08
4079        EL  9.7104

1000000

In [36]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    # Verificar si la columna 'm2_contruido' existe en el DataFrame
    #if 'm2_contruido' in df.columns:
        # Renombrar la columna
    df.rename(columns={'Metros_construido':'m2_construido','m2_contruido': 'm2_construido','Category':'categoria','segmento':'categoria','Categoria':'categoria','metros_construido':'m2_construido','Precio':'precio'}, inplace=True)
    # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")

Columnas renombradas en los DataFrames filtrados.


In [37]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
dataframes_name = []

for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['categoria', 'precio']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)
     # Componer el nombre del DataFrame en la lista dataframes_name
    dataframes_name.append(nombre_df)


In [39]:
# Diccionario para almacenar el total de precios por categoría
total_precios_por_segmento = {}

# Diccionario para almacenar el número de observaciones por categoría
num_observaciones_por_segmento = {}

# Función para extraer el segmento correctamente
def extraer_segmento(categoria):
    # Verificar que la categoría no sea nula o vacía
    if not isinstance(categoria, str) or len(categoria) == 0:
        return 'Desconocido'  # Asignar un valor por defecto si la categoría está vacía

    if categoria.startswith('EL') or categoria.startswith('ELITE'):
        return 'ELITE'  # Agrupar todas las categorías EL o ELITE como 'ELITE'
    elif categoria.startswith('E'):
        return categoria[0]  # Mantener la letra E para E1, E2, etc.
    else:
        return categoria[0]  # Para otras categorías, usar la primera letra

# Iterar sobre la lista de DataFrames
for df in dataframes_list:
    # Añadir una nueva columna 'segmento' aplicando la función extraer_segmento
    df['segmento'] = df['categoria'].apply(extraer_segmento)

    # Calcular el total de precios por segmento en el DataFrame actual
    for segmento, group in df.groupby('segmento'):
        total_precios_por_segmento[segmento] = total_precios_por_segmento.get(segmento, 0) + group['precio'].sum()
        num_observaciones_por_segmento[segmento] = num_observaciones_por_segmento.get(segmento, 0) + len(group)

# Diccionario para almacenar el promedio de precios por segmento
promedio_precios_por_segmento = {}

# Calcular el promedio de precios por segmento
for segmento in total_precios_por_segmento:
    promedio_precios_por_segmento[segmento] = total_precios_por_segmento[segmento] / num_observaciones_por_segmento[segmento]

# Convertir el diccionario a un DataFrame
promedio_precios_por_segmento_df = pd.DataFrame(list(promedio_precios_por_segmento.items()), columns=['segmento', 'promedio_precio'])

# Ver el DataFrame de promedios por segmento
promedio_precios_por_segmento_df


C:\Users\yoe11\AppData\Local\Temp\ipykernel_19040\4216386809.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_19040\4216386809.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_19040\4216386809.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

,segmento,promedio_precio
0,A,3.646810e+06
1,B,2.879546e+06
2,C,2.135439e+06
3,D,1.376424e+06
4,Desconocido,4.439728e+06
5,E,6.840395e+05
6,ELITE,3.720950e+07
7,L,1.547643e+07
8,S,6.270798e+06


In [40]:
# Definir el orden deseado para los segmentos
orden_segmentos=["E", "D", "C", "B","A", "S", "L", 'ELITE']
promedio_precios_por_segmento_df = promedio_precios_por_segmento_df[promedio_precios_por_segmento_df['segmento'].isin(orden_segmentos)]

# Convertir la columna 'segmento' a una categoría con el orden especificado
promedio_precios_por_segmento_df['segmento'] = pd.Categorical(promedio_precios_por_segmento_df['segmento'], 
                                                               categories=orden_segmentos, 
                                                               ordered=True)

# Ordenar el DataFrame según la columna 'segmento'
promedio_precios_por_segmento_df.sort_values('segmento', inplace=True)
promedio_precios_por_segmento_df

C:\Users\yoe11\AppData\Local\Temp\ipykernel_19040\894412361.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_19040\894412361.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,segmento,promedio_precio
5,E,6.840395e+05
3,D,1.376424e+06
2,C,2.135439e+06
1,B,2.879546e+06
0,A,3.646810e+06
8,S,6.270798e+06
7,L,1.547643e+07
6,ELITE,3.720950e+07


In [43]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']


fig = go.Figure()
# Agregar las barras al gráfico
fig.add_trace(go.Bar(
    x=promedio_precios_por_segmento_df['segmento'],
    y=promedio_precios_por_segmento_df['promedio_precio'],
    text=promedio_precios_por_segmento_df['promedio_precio'],
    texttemplate='%{text:.2s}',  # Mostrar los valores en las barras
    textposition='outside',
    marker_color=colores,  # Usar la paleta de colores definida
))
# Actualizar el diseño de la gráfica
fig.update_layout(
    #title='Promedio de precios por segmento',
    xaxis_title='Segmento',
    yaxis=dict(
        title='Promedio de Precio',
        gridcolor='#dddcda',  # Color de las líneas que dividen los rangos del eje Y
        gridwidth=1,
    ),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    legend_title='Segmento',
    legend=dict(
        orientation='h',  # Orientación horizontal de la leyenda
        yanchor='bottom',  # Anclar al borde inferior
        y=1.02,  # Colocar la leyenda justo debajo del gráfico
        xanchor='right',
        x=1
    ),
    margin=dict(l=10, r=10, t=50, b=10)  # Ajusta los márgenes (left, right, top, bottom)
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, os.path.join(carpeta, f'{nombre_archivo}.html'))

# Exportar
guardar_grafico_como_html(fig, 'g_bar_precio_promedio_segmento', carpeta='assets/graficas')
fig.show()